# 1. Données d'entrées

## 1.1. Les bibliothèques nécessaires et initialisation Spark

In [0]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import io
# io is necessary for scaling data
import boto3
# boto3 is necessary for importing the results to AWS S3


# PySpark libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import split, col, pandas_udf, PandasUDFType, udf
from pyspark.ml.linalg import Vectors, VectorUDT

# Transfer learning
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# MLlib, Spark’s Machine Learning (ML) library
from pyspark.ml.feature import StandardScaler, PCA

In [0]:
# Set up Hadoop Configurations for AWS_ACCESS_KEY & AWS_SECRET_KEY

ACCESS_KEY = "XXXX"

SECRET_KEY = "XXXXXXXX"

ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")

AWS_BUCKET_NAME = "projet-8-bucket"

MOUNT_NAME = "mount_projet-8-bucket"

# dbutils.fs.mount(f"s3a://{ACCESS_KEY}:{ENCODED_SECRET_KEY}@{AWS_BUCKET_NAME}", f"/mnt/{MOUNT_NAME}")

print('AWS S3 bucket {} is successfully mounted.'.format(AWS_BUCKET_NAME))

AWS S3 bucket projet-8-bucket is successfully mounted.

## 1.2. Téléchargement du jeux de données

In [0]:
# Téléchargement des images depuis AWS S3
# HOW TO : https://docs.databricks.com/data/data-sources/aws/amazon-s3.html
# Exemple : https://www.linkedin.com/pulse/readwrite-mount-from-aws-s3-databricks-deepak-rajak/

S3_URI = "/mnt/mount_projet-8-bucket/data/*"
data = spark.read.format("binaryfile").load(S3_URI)

In [0]:
display(data)

path,modificationTime,length,content
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_116.jpg,2022-02-18T13:31:46.000+0000,125373,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_114.jpg,2022-02-18T13:31:46.000+0000,125088,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_108.jpg,2022-02-18T13:31:46.000+0000,124905,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_118.jpg,2022-02-18T13:31:46.000+0000,124363,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_120.jpg,2022-02-18T13:31:46.000+0000,124026,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_142.jpg,2022-02-18T13:31:46.000+0000,123109,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_112.jpg,2022-02-18T13:31:46.000+0000,123094,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_106.jpg,2022-02-18T13:31:46.000+0000,122551,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_102.jpg,2022-02-18T13:31:46.000+0000,122525,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"
dbfs:/mnt/mount_projet-8-bucket/data/apple_hit_1/r0_144.jpg,2022-02-18T13:31:46.000+0000,122317,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgo= (truncated), ...)"


In [0]:
data.printSchema()

root
-- path: string (nullable = true)
-- modificationTime: timestamp (nullable = true)
-- length: long (nullable = true)
-- content: binary (nullable = true)

In [0]:
data = data.withColumn('label', split(col('path'), '/').getItem(4)).select('path', 'content','label')
data.show()

+--------------------+--------------------+-----------+
 path| content| label|
+--------------------+--------------------+-----------+
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
dbfs:/mnt/mount_p...|[FF D8 FF E0 00 1...|apple_hit_1|
+--------------------+--------------------+-----------+
only showing top 20 rows

# 2. Préparation du modèle de Transfer Learning

In [0]:
# Source : https://docs.databricks.com/_static/notebooks/deep-learning/deep-learning-transfer-learning-keras.html

## 2.1. Le modèle ResNet50

In [0]:
model = ResNet50(include_top=False)
model.summary()  # verify that the top layer is removed

Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
 16384/94765736 [..............................] - ETA: 0s 4202496/94765736 [>.............................] - ETA: 3s14106624/94765736 [===>..........................] - ETA: 1s23928832/94765736 [======>.......................] - ETA: 0s33562624/94765736 [=========>....................] - ETA: 1s38559744/94765736 [===========>..................] - ETA: 0s46063616/94765736 [=============>................] - ETA: 0s53190656/94765736 [===============>..............] - ETA: 0s58613760/94765736 [=================>............] - ETA: 0s65011712/94765736 [===================>..........] - ETA: 0s70877184/94765736 [=====================>........] - ETA: 0s77078528/94765736 [=======================>......] - ETA: 0s84590592/94765736 [=========================>....] - ETA: 0s90398720/94765736 [===========================>..] - ETA: 0s94773248/94765736 [==============================] - 1s 0us/step
94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type) Output Shape Param # Connected to 
==================================================================================================
 input_1 (InputLayer) [(None, None, None, 0 [] 
 3)] 
 
 conv1_pad (ZeroPadding2D) (None, None, None, 0 ['input_1[0][0]'] 
 3) 
 
 conv1_conv (Conv2D) (None, None, None, 9472 ['conv1_pad[0][0]'] 
 64) 
 
 conv1_bn (BatchNormalization) (None, None, None, 256 ['conv1_conv[0][0]'] 
 64) 
 
 conv1_relu (Activation) (None, None, None, 0 ['conv1_bn[0][0]'] 
 64) 
 
 pool1_pad (ZeroPadding2D) (None, None, None, 0 ['conv1_relu[0][0]'] 
 64) 
 
 pool1_pool (MaxPooling2D) (None, None, None, 0 ['pool1_pad[0][0]'] 
 64) 
 
 conv2_block1_1_conv (Conv2D) (None, None, None, 4160 ['pool1_pool[0][0]'] 
 64) 
 
 conv2_block1_1_bn (BatchNormal (None, None, None, 256 ['conv2_block1_1_conv[0][0]'] 
 ization) 64) 
 
 conv2_block1_1_relu (Activatio (None, None, None, 0 ['conv2_block1_1_bn[0][0]'] 
 n) 64) 
 
 conv2_block1_2_conv (Conv2D) (None, None, None, 36928 ['conv2_block1_1_relu[0][0]'] 
 64) 
 
 conv2_block1_2_bn (BatchNormal (None, None, None, 256 ['conv2_block1_2_conv[0][0]'] 
 ization) 64) 
 
 conv2_block1_2_relu (Activatio (None, None, None, 0 ['conv2_block1_2_bn[0][0]'] 
 n) 64) 
 
 conv2_block1_0_conv (Conv2D) (None, None, None, 16640 ['pool1_pool[0][0]'] 
 256) 
 
 conv2_block1_3_conv (Conv2D) (None, None, None, 16640 ['conv2_block1_2_relu[0][0]'] 
 256) 
 
 conv2_block1_0_bn (BatchNormal (None, None, None, 1024 ['conv2_block1_0_conv[0][0]'] 
 ization) 256) 
 
 conv2_block1_3_bn (BatchNormal (None, None, None, 1024 ['conv2_block1_3_conv[0][0]'] 
 ization) 256) 
 
 conv2_block1_add (Add) (None, None, None, 0 ['conv2_block1_0_bn[0][0]', 
 256) 'conv2_block1_3_bn[0][0]'] 
 
 conv2_block1_out (Activation) (None, None, None, 0 ['conv2_block1_add[0][0]'] 
 256) 
 
 conv2_block2_1_conv (Conv2

In [0]:
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
  """
  Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
  """
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model

## 2.2. Definir image loading and featurization logic via Pandas UDF

In [0]:
def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content)).resize([50, 50])
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

In [0]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
 warnings.warn(

In [0]:
# Application des fonctions définis

features_df = data.select(col("path"), col("label"), featurize_udf("content").alias("features"))
features_df.printSchema()
features_df.show()

root
-- path: string (nullable = true)
-- label: string (nullable = true)
-- features: array (nullable = true)
 |-- element: float (containsNull = true)

+--------------------+-----------+--------------------+
 path| label| features|
+--------------------+-----------+--------------------+
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 6.8223476, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 7.0397773, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 4.6407127, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 6.201755, 2...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 7.0180907, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.1638732, 0.105...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 5.630745, 0...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 4.8694057, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 3.9284728, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.64675486, 0.53...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 4.1014285, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[3.759369, 0.0, 1...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 1.8148413, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 4.1946926, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 4.3345146, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[3.680339, 0.0, 1...|
dbfs:/mnt/mount_p...|apple_hit_1|[3.6866057, 0.0, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 1.7578216, ...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 0.03485298,...|
dbfs:/mnt/mount_p...|apple_hit_1|[0.0, 3.682842, 1...|
+--------------------+-----------+--------------------+
only showing top 20 rows

In [0]:
# Modification du type des données

features_vector = udf(lambda l: Vectors.dense(l), VectorUDT())
features_df = features_df.select(col("path"),  col("label"), features_vector(features_df["features"]).alias("features"))

# 3. Reduction dimensionelle via PCA

In [0]:
# Normalisation des données

scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
)

# Fit
scaler_fit = scaler.fit(features_df)

# Transform
features_df_scaled = scaler_fit.transform(features_df)

In [0]:
# Réalisation d'une ACP

n_components = 16
pca = PCA(
    k = n_components, 
    inputCol = 'scaledFeatures', 
    outputCol = 'pcaFeatures'
).fit(features_df_scaled)

data_pca = pca.transform(features_df_scaled)

# print('Explained Variance Ratio', pca.explainedVariance.toArray())
# data_pca.show(2)

#4. Exportation des resultats à S3

In [0]:
# Transformation du data via Pandas

data_pandas = data_pca.select('path', 'label', 'pcaFeatures').toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field pcaFeatures. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
# Exportation to S3 bucket

from io import StringIO

bucket = "projet-8-bucket" 

csv_buffer = StringIO()
data_pandas.to_csv(csv_buffer)

s3_resource = boto3.resource('s3', aws_access_key_id=ACCESS_KEY,
         aws_secret_access_key= SECRET_KEY)

s3_resource.Object(bucket, 'data_pandas_pca.csv').put(Body=csv_buffer.getvalue())

Out[20]: {'ResponseMetadata': {'RequestId': 'SVF7XW5CYKPMJQ5V',
 'HostId': 'szVDsAyqziU89ZYwSt6FTi4D2ro0rwl90NtcTqOKEO9yEO3qbAG7ybT7CS1uYfzcpvbRd+RyWw0=',
 'HTTPStatusCode': 200,
 'HTTPHeaders': {'x-amz-id-2': 'szVDsAyqziU89ZYwSt6FTi4D2ro0rwl90NtcTqOKEO9yEO3qbAG7ybT7CS1uYfzcpvbRd+RyWw0=',
 'x-amz-request-id': 'SVF7XW5CYKPMJQ5V',
 'date': 'Sun, 20 Feb 2022 12:35:59 GMT',
 'etag': '"280a34c3f649fbf66bb866d69a662949"',
 'server': 'AmazonS3',
 'content-length': '0'},
 'RetryAttempts': 1},
 'ETag': '"280a34c3f649fbf66bb866d69a662949"'}